# Optimizing Models for Serving

## Build the Graph Optimization Tool

The full tools build takes ~ 20 minutes.  Start at the beginning of class!

In [ ]:
%%bash
# install bazel
wget https://github.com/bazelbuild/bazel/releases/download/0.16.0/bazel-0.16.0-installer-linux-x86_64.sh
chmod +x bazel-0.16.0-installer-linux-x86_64.sh
./bazel-0.16.0-installer-linux-x86_64.sh --user
export PATH=$HOME/bin:$PATH

# build gtt and summarize graph
git clone https://github.com/tensorflow/tensorflow
cd tensorflow
git checkout r1.9
bazel build tensorflow/tools/graph_transforms:transform_graph
bazel build tensorflow/tools/graph_transforms:summarize_graph

## Download the SavedModel

In [ ]:
%%bash
gsutil cp gs://tfs-data/mnist_cnn_model.tgz .
tar xzf mnist_cnn_model.tgz

## Inspect the model

In [ ]:
%%bash
tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=mnist_cnn_model/graph.pbtxt

## Run the Graph Optimization Tool on the model

In [ ]:
%%bash

tensorflow/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
--in_graph=mnist_cnn_model/graph.pbtxt \
--out_graph=mnist_cnn_model/optimized_model.pb \
--inputs='enqueue_input/Placeholder' \
--outputs='softmax' \
--transforms='
strip_unused_nodes(type=float, shape="1,28,28,1")
remove_nodes(op=Identity, op=CheckNumerics)
'

## Inspect the model again

In [ ]:
%%bash
tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=mnist_cnn_model/graph.pbtxt

## Freeze the graph

In [ ]:
%%bash
tensorflow/bazel-bin/tensorflow/python/tools/freeze_graph \
--input_graph=mnist_cnn_model/optimized_model.pb \
--input_checkpoint=mnist_cnn_model/model.ckpt-20000 \
--output_graph=mnist_cnn_model/frozen_graph.pb \
--output_node_names=softmax

In [ ]:
%%bash
tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=mnist_cnn_model/frozen_graph.pb